<a href="https://colab.research.google.com/github/GuilhermeOrtega/GuilhermeOrtega/blob/main/Regress%C3%A3o_Log%C3%ADstica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# AULA PRÁTICA DE REGRESSÃO LOGÍSTICA - MODELO BASELINE
# Objetivo: Prever a probabilidade de sobrevivência no Titanic
# ==============================================================================

# 1. IMPORTAÇÃO DAS BIBLIOTECAS
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# ------------------------------------------------------------------------------

# 2. CARREGAMENTO E PREPARAÇÃO DOS DADOS
# Usaremos o Seaborn para carregar o dataset, o que facilita a execução em qualquer ambiente.
print("--- Carregando e Preparando os Dados ---")
df = sns.load_dataset('titanic')
df

--- Carregando e Preparando os Dados ---


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


Dicionário de Dados: Titanic (Formato Texto)
survived: (Categórica, Binária) - A nossa variável alvo. Indica se o passageiro sobreviveu, onde 0 representa "Não" e 1 representa "Sim".

pclass: (Categórica, Ordinal) - A classe socioeconômica do passageiro. Os valores são 1 para Primeira Classe, 2 para Segunda Classe e 3 para Terceira Classe. A ordem aqui importa.

sex: (Categórica, Binária) - O sexo do passageiro, registrado como "male" (masculino) ou "female" (feminino).

age: (Numérica, Contínua) - A idade do passageiro em anos. Esta coluna possui valores fracionários para crianças com menos de um ano e também contém dados ausentes.

sibsp: (Numérica, Discreta) - Refere-se ao número de irmãos (siblings) ou cônjuges (spouses) que o passageiro tinha a bordo do navio.

parch: (Numérica, Discreta) - Refere-se ao número de pais (parents) ou filhos (children) que o passageiro tinha a bordo.

ticket: (Alfanumérica) - O número do bilhete do passageiro. É uma variável com formato livre, podendo conter letras e números, o que a torna complexa para uso direto em modelos.

fare: (Numérica, Contínua) - O valor da tarifa que o passageiro pagou pelo bilhete.

cabin: (Alfanumérica) - O número da cabine do passageiro. Esta é a coluna com a maior quantidade de dados ausentes.

embarked: (Categórica) - O porto onde o passageiro embarcou. Os valores possíveis são C (Cherbourg), Q (Queenstown) e S (Southampton).

In [3]:
# Para um modelo baseline, vamos selecionar apenas as colunas mais relevantes
# e remover as que têm muitos dados faltantes (como 'deck') ou são complexas (como 'Name').
colunas_selecionadas = ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
df = df[colunas_selecionadas]
df

,survived,pclass,sex,age,sibsp,parch,fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000
887,1,1,female,19.0,0,0,30.0000
888,0,3,female,NaN,1,2,23.4500
889,1,1,male,26.0,0,0,30.0000


In [4]:
# Tratamento simples de dados faltantes:
# Preencher a idade com o valor da mediana (robusto a outliers)
df['age'] = df['age'].fillna(df['age'].median())

/tmp/ipython-input-2873738127.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].fillna(df['age'].median())


In [5]:
# Transformação de variável categórica:
# Converter a coluna 'sex' para valores numéricos (0 para 'male', 1 para 'female')
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

/tmp/ipython-input-1798999199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'] = df['sex'].map({'male': 0, 'female': 1})


In [6]:
print("Dados após a limpeza:")
df.head()

Dados após a limpeza:


,survived,pclass,sex,age,sibsp,parch,fare
0,0,3,0,22.0,1,0,7.2500
1,1,1,1,38.0,1,0,71.2833
2,1,3,1,26.0,0,0,7.9250
3,1,1,1,35.0,1,0,53.1000
4,0,3,0,35.0,0,0,8.0500


In [7]:
print("\nInformações do DataFrame:")
df.info()


Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    int64  
 3   age       891 non-null    float64
 4   sibsp     891 non-null    int64  
 5   parch     891 non-null    int64  
 6   fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [8]:
# 3. DEFINIÇÃO DAS FEATURES (X) E DO ALVO (y)
# X contém as variáveis que usaremos para prever.
# y é a variável que queremos prever (0 = Não sobreviveu, 1 = Sobreviveu).
X = df.drop('survived', axis=1)
y = df['survived']

In [9]:
X

,pclass,sex,age,sibsp,parch,fare
0,3,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,3,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,3,0,35.0,0,0,8.0500
...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000
887,1,1,19.0,0,0,30.0000
888,3,1,28.0,1,2,23.4500
889,1,0,26.0,0,0,30.0000


In [10]:
y

,survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [11]:
# 4. DIVISÃO EM DADOS DE TREINO E DE TESTE
# Separamos 70% dos dados para treinar o modelo e 30% para testar sua performance.
# random_state=42 garante que a divisão seja sempre a mesma, para reprodutibilidade.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
X_train

,pclass,sex,age,sibsp,parch,fare
445,1,0,4.0,0,2,81.8583
650,3,0,28.0,0,0,7.8958
172,3,1,1.0,1,1,11.1333
450,2,0,36.0,1,2,27.7500
314,2,0,43.0,1,1,26.2500
...,...,...,...,...,...,...
106,3,1,21.0,0,0,7.6500
270,1,0,28.0,0,0,31.0000
860,3,0,41.0,2,0,14.1083
435,1,1,14.0,1,2,120.0000


In [13]:
X_test

,pclass,sex,age,sibsp,parch,fare
709,3,0,28.0,1,1,15.2458
439,2,0,31.0,0,0,10.5000
840,3,0,20.0,0,0,7.9250
720,2,1,6.0,0,1,33.0000
39,3,1,14.0,1,0,11.2417
...,...,...,...,...,...,...
821,3,0,27.0,0,0,8.6625
633,1,0,28.0,0,0,0.0000
456,1,0,65.0,0,0,26.5500
500,3,0,17.0,0,0,8.6625


In [14]:
# 5. CRIAÇÃO E TREINAMENTO DO MODELO DE REGRESSÃO LOGÍSTICA
# Instanciamos o modelo. max_iter=1000 ajuda a garantir que o modelo convirja.
model = LogisticRegression(max_iter=1000)
model

LogisticRegression(max_iter=1000)

In [15]:
# Treinamos o modelo com os dados de treino.
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [16]:
# 6. REALIZAÇÃO DAS PREVISÕES E AVALIAÇÃO DO MODELO
# O modelo agora usa os dados de teste (que ele nunca viu) para fazer previsões.
predictions = model.predict(X_test)
predictions

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0])

In [17]:
# Comparamos as previsões com os resultados reais (y_test) para avaliar.
print("--- Avaliação do Modelo ---")
print("\nMatriz de Confusão:")
# Mostra os acertos e erros (VN, FP, FN, VP)
print(confusion_matrix(y_test, predictions))

--- Avaliação do Modelo ---

Matriz de Confusão:
[[139  18]
 [ 32  79]]


1. O Quadrante dos Acertos
139 - Verdadeiros Negativos (VN):

O que significa: Foram 139 passageiros que não sobreviveram, e o seu modelo previu corretamente que eles não sobreviveriam.

Em resumo: 139 acertos ao prever as mortes. Isso é ótimo!

79 - Verdadeiros Positivos (VP):

O que significa: Foram 79 passageiros que sobreviveram, e o seu modelo previu corretamente que eles sobreviveriam.

Em resumo: 79 acertos ao prever as sobrevivências. Isso também é ótimo!

Total de Acertos: 139+79=218 passageiros previstos corretamente.

2. O Quadrante dos Erros (O mais importante para analisar!)
18 - Falsos Positivos (FP) - Erro Tipo I:

O que significa: Foram 18 passageiros que na realidade não sobreviveram, mas o modelo errou e previu que eles iriam sobreviver.

Em resumo: O modelo teve um "otimismo" equivocado 18 vezes, dando falsas esperanças.

32 - Falsos Negativos (FN) - Erro Tipo II:

O que significa: Foram 32 passageiros que na realidade sobreviveram, mas o modelo errou e previu que eles iriam morrer.

Em resumo: O modelo foi "pessimista" e errou 32 vezes, prevendo uma tragédia que não aconteceu para essas pessoas. Este é frequentemente o tipo de erro mais perigoso em outros cenários (como um diagnóstico médico errado).

Total de Erros: 18+32=50 passageiros previstos incorretamente.

In [18]:
print("\nRelatório de Classificação:")
# Mostra as métricas de Precisão, Recall, F1-Score e Acurácia
print(classification_report(y_test, predictions))


Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       157
           1       0.81      0.71      0.76       111

    accuracy                           0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268



In [19]:
# 7. INTERPRETAÇÃO DOS COEFICIENTES (ODDS RATIO)
print("--- Interpretação dos Coeficientes ---")
# Os coeficientes do modelo estão em log-odds. Precisamos convertê-los.
# Usamos a função exponencial (np.exp) para obter o Odds Ratio.
odds_ratios = np.exp(model.coef_[0])

--- Interpretação dos Coeficientes ---


In [20]:

# Criamos um DataFrame para visualizar a importância de cada variável.
coeff_df = pd.DataFrame(odds_ratios, X.columns, columns=['Odds Ratio'])


In [21]:
print("DataFrame de Odds Ratio:")
print(coeff_df)
print("\nLembre-se: Para cada aumento de 1 unidade na variável, a CHANCE (odds) de sobreviver é MULTIPLICADA pelo valor do Odds Ratio.")

DataFrame de Odds Ratio:
        Odds Ratio
pclass    0.407397
sex      12.244933
age       0.966490
sibsp     0.748561
parch     0.894687
fare      1.003756

Lembre-se: Para cada aumento de 1 unidade na variável, a CHANCE (odds) de sobreviver é MULTIPLICADA pelo valor do Odds Ratio.


A Regra de Interpretação
Antes de começar, vamos estabelecer a base:

Odds Ratio > 1: Aumenta a chance (odds) de sobreviver.

Odds Ratio < 1: Diminui a chance (odds) de sobreviver.

Odds Ratio = 1: Não tem efeito na chance de sobreviver.

Análise Detalhada do DataFrame de Odds Ratio

sex (Odds Ratio: 12.24)
Este é, de longe, o fator mais impactante.

O que significa "aumento de 1 unidade"? No nosso código, male é 0 e female é 1. Um aumento de 1 unidade significa "mudar de homem para mulher".

Interpretação: Mantendo todas as outras variáveis constantes, ser do sexo feminino multiplica a chance (odds) de sobreviver por 12.24. Em outras palavras, o modelo aprendeu que as mulheres tinham uma chance de sobrevivência drasticamente maior que os homens, confirmando a regra histórica de "mulheres e crianças primeiro".

pclass (Odds Ratio: 0.407)
Este é o segundo fator mais forte, mas com efeito negativo.

O que significa "aumento de 1 unidade"? Significa descer de classe (ir da 1ª para a 2ª, ou da 2ª para a 3ª).

Interpretação: Para cada classe que um passageiro "desce", sua chance de sobreviver é multiplicada por 0.407. Isso equivale a uma redução de quase 60% na chance de sobreviver (1−0.407=0.593). Fica claro que passageiros da primeira classe tinham uma vantagem imensa.

age (Odds Ratio: 0.966)
Um efeito negativo, porém mais sutil.

O que significa "aumento de 1 unidade"? Significa ficar um ano mais velho.

Interpretação: Para cada ano a mais na idade de um passageiro, sua chance de sobreviver é multiplicada por 0.966. Isso representa uma pequena redução de 3.4% na chance de sobrevivência por ano de vida. O modelo capturou uma leve tendência de que passageiros mais velhos tiveram mais dificuldade para sobreviver.

sibsp (Odds Ratio: 0.748)
Um efeito negativo moderado.

O que significa "aumento de 1 unidade"? Ter um irmão ou cônjuge a mais a bordo.

Interpretação: Para cada irmão/cônjuge a mais, a chance de sobreviver é multiplicada por 0.748, representando uma redução de 25.2%. O modelo sugere que viajar com muitos irmãos ou com um cônjuge pode ter diminuído as chances de sobrevivência, talvez por tentarem ficar juntos.

parch (Odds Ratio: 0.894)
Um efeito negativo mais fraco.

O que significa "aumento de 1 unidade"? Ter um pai ou filho a mais a bordo.

Interpretação: Para cada pai ou filho a mais, a chance de sobreviver é multiplicada por 0.894, uma redução de 10.6%. O efeito é similar ao de sibsp, mas menos pronunciado.

fare (Odds Ratio: 1.0037)
Um efeito positivo quase nulo.

O que significa "aumento de 1 unidade"? Pagar um dólar a mais na tarifa do bilhete.

Interpretação: Para cada dólar a mais pago na tarifa, a chance de sobreviver aumenta em apenas 0.37%. Por que um efeito tão pequeno, se sabemos que quem pagou mais (1ª classe) sobreviveu mais? Porque a variável pclass já captura essa informação de forma muito mais forte. O modelo percebe que, uma vez que você já sabe a classe do passageiro, o valor exato da tarifa adiciona muito pouca informação nova.

Hierarquia de Importância (O que o modelo aprendeu)
Ser mulher (sex) foi o fator mais decisivo para aumentar a chance de sobrevivência.

Estar em uma classe mais alta (pclass) foi o segundo fator mais importante.

Ser mais jovem (age) e viajar com menos familiares (sibsp, parch) também ajudaram, mas com menor intensidade.

O valor da tarifa (fare) teve um impacto quase desprezível, pois sua informação já estava contida na classe do passageiro.